In [2]:
from scamp import test_run, Session, settings, wait
from config import ARTIFACTS_RELPATH
from scamp_utils import set_default_scamp_settings
from scamp_extensions.pitch import Scale, ScaleType, hertz_to_midi
from numbers import Real
from operator import itemgetter
from itertools import chain

set_default_scamp_settings()

ALSA lib conf.c:4005:(snd_config_hooks_call) Cannot open shared library libasound_module_conf_pulse.so (/opt/conda/envs/default/lib/alsa-lib/libasound_module_conf_pulse.so: cannot open shared object file: No such file or directory)
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default
fluidsynth: error: Failed to open the "default" audio device
ALSA lib seq_hw.c:466:(snd_seq_hw_open) open /dev/snd/seq failed: No such file or directory
fluidsynth: error: Error opening ALSA sequencer
fluidsynth: error: Couldn't find the requested audio driver 'coreaudio'.
fluidsynth: This build of fluidsynth supports the following audio drivers: alsa, file, jack, oss, pulseaudio, sdl2
ALSA lib seq_hw.c:466:(snd_seq_hw_open) open /dev/snd/seq failed: No such file or directory
fluidsynth: error: Error opening ALSA sequencer
fluidsynth: error: Couldn't find the requested audio driver 'dsound'.
fluidsynth: This build of fluidsynth supports the following audio drivers: alsa, file, jack, oss, pulseau

Testing for working audio driver...
jackdmp 1.9.22
Copyright 2001-2005 Paul Davis and others.
Copyright 2004-2016 Grame.
Copyright 2016-2023 Filipe Coelho.
jackdmp comes with ABSOLUTELY NO WARRANTY
This is free software, and you are welcome to redistribute it
under certain conditions; see the file COPYING for details


Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
fluidsynth: error: Failed to connect to Jack server.
ALSA lib seq_hw.c:466:(snd_seq_hw_open) open /dev/snd/seq failed: No such file or directory
fluidsynth: error: Error opening ALSA sequencer
fluidsynth: error: Couldn't find the requested audio driver 'portaudio

In [3]:
supercollider_session = Session().run_as_server()

In [40]:
phased_osc = supercollider_session.new_osc_part(
    "phased", port=57120, ip_address="supercollider"
)
mono_osc = supercollider_session.new_osc_part(
    "mono", port=57120, ip_address="supercollider"
)

# supercollider_session.tempo = 120
# chord = [60, 64, 67, 70]
# sequence = [map(lambda x: x+ i*12, chord) for i in range(2)]

scale = Scale.dorian(60)  # type: ignore


def chord(start: int) -> tuple[float, float, float]:
    steps = [(s + start) % 12 for s in [0, 3, 5, 7]]
    return itemgetter(*steps)(scale)


chords = [chord(root) for root in range(8)]
# root = triad(0)
# fourth = triad(4)
# fifth = triad(5)

sequence = list(chain(*[chord(root) for root in [0, 7, 4, 2]])) * 2

# for freq in scale:
#     print(freq)
# chord = [0, 5, 7, 11]
# sequence = []
# for i in scale:
#     for voice in map(lambda x: x + i*12 + 60, chord):
#         sequence.append(voice)
# sequence = [ for i in range(3)]

# print(sequence)

for pitch in sequence:
    mono_osc.play_note(
        pitch,
        0.2,
        0.2,
        {"param_fadeTime": 0.1},
        # blocking=False
    )
    # phased_osc.play_note(pitch, 0.7, 0.3, {"param_tremoloFreq": 0.1})

[0, 3, 5, 7]
[1, 4, 6, 8]
[2, 5, 7, 9]
[3, 6, 8, 10]
[4, 7, 9, 11]
[5, 8, 10, 0]
[6, 9, 11, 1]
[7, 10, 0, 2]
[0, 3, 5, 7]
[7, 10, 0, 2]
[4, 7, 9, 11]
[2, 5, 7, 9]


In [45]:
course_osc = supercollider_session.new_osc_part(
    "course", port=57120, ip_address="supercollider"
)


def seventh(mode: str, pitch: int) -> list[int]:
    return itemgetter(0, 3, 5, 7)(getattr(Scale, mode)(pitch))


root = 60

progression1 = [
    seventh("ionian", 60),
    seventh("aeolian", 69),
    seventh("mixolydian", 67),
    seventh("dorian", 62),
] * 2

progression2 = [
    seventh("locrian", 71),
    seventh("lydian", 65),
    seventh("phrygian", 64),
    seventh("aeolian", 69),
] * 2

progression = [
    *progression1,
    # *progression2
]

strum_speed = 0.07
drift = 0.4
fade = 0.2


for chord in progression:
    for pitch in chord:
        course_osc.play_note(
            pitch,
            1 / len(chord),
            0.5,
            {
                "param_drift": drift,
                "param_fadeTime": fade,
            },
        )
        wait(strum_speed)
    wait(1 - len(chord) * strum_speed)